In [1]:
# ================================================
# IMPORT COMPLET DES DONNÉES - PARIS - VERSION FINALE
# Projet: Spéculation immobilière vs Accès au logement
# ================================================

print("DÉMARRAGE DE L'IMPORT DES DONNÉES")
print("=" * 70)
print("Projet: Analyse spéculation immobilière / accès au logement - Paris")
print("=" * 70)

# ----------------------------------------------------
# 1. CONFIGURATION INITIALE
# ----------------------------------------------------

print("\n" + "=" * 70)
print("PHASE 1: Configuration de l'environnement...")
print("=" * 70)

import pandas as pd
import numpy as np
import geopandas as gpd
import requests
import zipfile
import io
from pathlib import Path
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Définition des chemins
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "data"
RAW_DIR = DATA_DIR / "raw"
PROC_DIR = DATA_DIR / "processed"
EXT_DIR = DATA_DIR / "external"

# Création des dossiers
print("Création de l'arborescence...")
for dossier in [DATA_DIR, RAW_DIR, PROC_DIR, EXT_DIR]:
    dossier.mkdir(parents=True, exist_ok=True)
    print(f"  ✓ {dossier}")

print("Configuration terminée")

# ----------------------------------------------------
# 2. TÉLÉCHARGEMENT DONNÉES DVF (Transactions immobilières)
# ----------------------------------------------------
print("\n" + "=" * 70)
print("PHASE 2: Téléchargement des données DVF pour Paris")
print("=" * 70)

print("URL: https://files.data.gouv.fr/geo-dvf/latest/csv/2023/departements/75.csv.gz")
print("Cette opération peut prendre quelques minutes...")

try:
    # Téléchargement direct du fichier Paris (département 75)
    dvf_url = "https://files.data.gouv.fr/geo-dvf/latest/csv/2023/departements/75.csv.gz"
    
    print("Téléchargement en cours...")
    df_dvf_full = pd.read_csv(dvf_url, compression='gzip', low_memory=False)
    
    print(f"✓ Données brutes téléchargées: {len(df_dvf_full):,} transactions")
    
    # Filtrage pour Paris intramuros (arrondissements 75101 à 75120)
    print("Filtrage pour Paris intramuros...")
    df_dvf_full['code_commune'] = df_dvf_full['code_commune'].astype(str)     
    codes_paris = [f'751{str(i).zfill(2)}' for i in range(1, 21)]             # création des codes insee de Paris (de 75101 à 75120)
    df_dvf = df_dvf_full[df_dvf_full['code_commune'].isin(codes_paris)].copy()   # création d'un dataframe contenant uniquement les données pour les arrondissements de Paris
    
    # Ajout des colonnes essentielles
    df_dvf['arrondissement'] = df_dvf['code_commune'].str[-2:].astype(int)   # cration de la variable "arrondissement" qui extrait les deux derniers caractères du code commune (et supprime le 0 si < 10)


    if 'date_mutation' in df_dvf.columns:
        df_dvf['date_mutation'] = pd.to_datetime(df_dvf['date_mutation'], errors='coerce') # Transformation de la colonne date_mutation en un format datetime de pandas
    
    if all(col in df_dvf.columns for col in ['valeur_fonciere', 'surface_reelle_bati']):
        df_dvf['prix_m2'] = df_dvf['valeur_fonciere'] / df_dvf['surface_reelle_bati']      # variable crée : prix_m2 = valeur fonciere / surface_reelle_bati
    
    # Sauvegarde
    df_dvf.to_csv(RAW_DIR / "dvf_paris_intramuros.csv", index=False)
    print(f"SUCCÈS: {len(df_dvf):,} transactions Paris sauvegardées")
    print(f"  Période: {df_dvf['date_mutation'].min().date() if 'date_mutation' in df_dvf.columns else 'N/A'}")
    print(f"  Arrondissements: {sorted(df_dvf['arrondissement'].unique())}")
    
except Exception as e:
    print(f"!!! ERREUR CRITIQUE lors du téléchargement DVF: {e}")
    print("Le programme s'arrête.")
    sys.exit(1)

# ----------------------------------------------------
# 3. TÉLÉCHARGEMENT DONNÉES INSEE (Arrondissements Paris)
# ----------------------------------------------------
print("\n" + "=" * 70)
print("PHASE 3: Téléchargement des données INSEE pour Paris")
print("=" * 70)

print("URL: https://www.insee.fr/fr/statistiques/fichier/5359146/dossier_complet.zip")
print("Cette archive contient les indicateurs socio-économiques par arrondissement...")

try:
    # Téléchargement de l'archive INSEE
    insee_url = "https://www.insee.fr/fr/statistiques/fichier/5359146/dossier_complet.zip"
    
    print("Téléchargement de l'archive...")
    response = requests.get(insee_url)
    
    if response.status_code == 200:
        print("Extraction des fichiers...")
        zip_content = io.BytesIO(response.content)
        
        with zipfile.ZipFile(zip_content) as z:
            # Vérification du contenu
            file_list = z.namelist()
            print(f"Fichiers dans l'archive: {file_list}")
            
            # Extraction du fichier principal
            if 'dossier_complet.csv' in file_list:
                with z.open('dossier_complet.csv') as f:
                    df_insee_full = pd.read_csv(f, sep=';', encoding='utf-8', low_memory=False)
                print(f"✓ Fichier principal chargé: {df_insee_full.shape}")
            else:
                print("!!! Fichier 'dossier_complet.csv' non trouvé dans l'archive")
                sys.exit(1)
        
        # Filtrage pour Paris (codes 75101 à 75120)
        print("Filtrage pour les arrondissements de Paris...")
        df_insee_full['CODGEO'] = df_insee_full['CODGEO'].astype(str)
        df_insee = df_insee_full[df_insee_full['CODGEO'].str.startswith('751')].copy()
        
        # Ajout du numéro d'arrondissement (pour pouvoir ensuite notamment croiser avec la table DVF des données financières)
        df_insee['arrondissement'] = df_insee['CODGEO'].str[-2:].astype(int)
        
        # Sauvegarde
        df_insee.to_csv(RAW_DIR / "insee_arrondissements_paris.csv", index=False)
        print(f"SUCCÈS: {len(df_insee)} arrondissements INSEE sauvegardés")
        print(f"  Variables disponibles: {len(df_insee.columns)}")
        
    else:
        print(f"!!! ERREUR HTTP {response.status_code} lors du téléchargement INSEE")
        sys.exit(1)
        
except Exception as e:
    print(f"!!! ERREUR lors du traitement INSEE: {e}")
    sys.exit(1)

# ----------------------------------------------------
# 4. TÉLÉCHARGEMENT CONTOURS GÉOGRAPHIQUES
# ----------------------------------------------------
print("\n" + "=" * 70)
print("PHASE 4: Téléchargement des contours géographiques")
print("=" * 70)

print("URL: https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/arrondissements/exports/geojson")

try:
    geo_url = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/arrondissements/exports/geojson?lang=fr"
    
    print("Téléchargement des polygones...")
    gdf_geo = gpd.read_file(geo_url)
    
    # Standardisation
    if 'l_ar' in gdf_geo.columns:
        gdf_geo['arrondissement'] = gdf_geo['l_ar'].str.extract(r'(\d+)').astype(int)
    
    # Sauvegarde
    gdf_geo.to_file(EXT_DIR / "arrondissements_paris.geojson", driver='GeoJSON')
    print(f"SUCCÈS: {len(gdf_geo)} arrondissements géographiques sauvegardés")
    print(f"  CRS: {gdf_geo.crs}")
    
except Exception as e:
    print(f"AVERTISSEMENT: Erreur lors du téléchargement géographique: {e}")
    print("Le projet peut continuer, mais les cartographies seront limitées.")
    gdf_geo = None

# ----------------------------------------------------
# 5. CHARGEMENT ET VÉRIFICATION DES DONNÉES
# ----------------------------------------------------
print("\n" + "=" * 70)
print("PHASE 5: Chargement et vérification finale")
print("=" * 70)

# Chargement depuis les fichiers sauvegardés
print("Chargement des données DVF...")
df_dvf = pd.read_csv(RAW_DIR / "dvf_paris_intramuros.csv", low_memory=False)
if 'date_mutation' in df_dvf.columns:
    df_dvf['date_mutation'] = pd.to_datetime(df_dvf['date_mutation'])

print("Chargement des données INSEE...")
df_insee = pd.read_csv(RAW_DIR / "insee_arrondissements_paris.csv", low_memory=False)

if gdf_geo is None and (EXT_DIR / "arrondissements_paris.geojson").exists():
    print("Chargement des contours depuis le fichier...")
    gdf_geo = gpd.read_file(EXT_DIR / "arrondissements_paris.geojson")

# Vérification de base
print("\nVÉRIFICATION RÉCAPITULATIVE:")
print(f"1. DVF: {len(df_dvf):,} transactions immobilières")
print(f"2. INSEE: {len(df_insee)} arrondissements, {len(df_insee.columns)} indicateurs")
print(f"3. GÉO: {len(gdf_geo) if gdf_geo is not None else 0} polygones")


DÉMARRAGE DE L'IMPORT DES DONNÉES
Projet: Analyse spéculation immobilière / accès au logement - Paris

PHASE 1: Configuration de l'environnement...
Création de l'arborescence...
  ✓ /home/onyxia/work/Speculation-immobiliere-et-acces-au-logement/notebooks/ # Notebooks d'analyse (à exécuter dans l'ordre)/data
  ✓ /home/onyxia/work/Speculation-immobiliere-et-acces-au-logement/notebooks/ # Notebooks d'analyse (à exécuter dans l'ordre)/data/raw
  ✓ /home/onyxia/work/Speculation-immobiliere-et-acces-au-logement/notebooks/ # Notebooks d'analyse (à exécuter dans l'ordre)/data/processed
  ✓ /home/onyxia/work/Speculation-immobiliere-et-acces-au-logement/notebooks/ # Notebooks d'analyse (à exécuter dans l'ordre)/data/external
Configuration terminée

PHASE 2: Téléchargement des données DVF pour Paris
URL: https://files.data.gouv.fr/geo-dvf/latest/csv/2023/departements/75.csv.gz
Cette opération peut prendre quelques minutes...
Téléchargement en cours...
✓ Données brutes téléchargées: 80,964 transac